In [1]:
import pandas as pd
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from csv import writer
from functools import reduce
pd.set_option('display.max_colwidth', -1)

## Access Azure storage table and blob

In [2]:
from azure.storage.blob import BlockBlobService
from azure.cosmosdb.table.tableservice import TableService
from azure.cosmosdb.table.models import Entity
import os
from bs4 import BeautifulSoup

In [3]:
from azure.storage.blob import (
    ContentSettings,
    BlobBlock,
    BlockListType,
)

In [4]:
block_blob_service = BlockBlobService(account_name='********', account_key='WRD*********8mtg==')
block_blob_service.MAX_CHUNK_GET_SIZE = 104857600
generator = block_blob_service.list_blobs('htmlarchive')

In [60]:
print(block_blob_service.MAX_CHUNK_GET_SIZE)
print(block_blob_service.MAX_SINGLE_GET_SIZE)

104857600
33554432


In [144]:
blob_list_0 = []
limit = 4

for blob in generator:
    #get the blob
    b = block_blob_service.get_blob_to_text('htmlarchive', blob.name)

    soup = BeautifulSoup(b.content, 'html.parser')
    ads = soup.find_all('p')[2].get_text()
    blob_list_0.append(ads)
    limit = limit -1
    
    if (limit == 0):
        break

In [ ]:
## get Urls 

In [29]:
def from_azure_to_URL(generator):
    '''Create a list and fill it with the database dictionary values.
    Return a pd DataFrame'''
    chunk_size  = 10000
    blob_list_0 = [] #this is the buffer
    chunk = 0 #this is the chunk number
    for (i,blob) in enumerate(generator): # i is the counter
        b = block_blob_service.get_blob_to_text('htmlarchive', blob.name)
        
        soup = BeautifulSoup(b.content, 'html.parser')
        links = soup.findAll('link', rel='canonical')
        for link in links:
            blob_list_0.append(link['href'])
        if i % chunk_size == 0: # after one chunck is done
            table = pd.DataFrame(blob_list_0)
            table.to_csv('URL_fromblob_jan28.{}.csv'.format(chunk))
            blob_list_0 = []
            chunk +=1
            print('chunk #',chunk,' is done')

    return table

In [32]:
#URL_list_unlabel = pd.DataFrame(blob_list_0)
URL_list_unlabel.to_pickle('URL_canonical_fromblob_jan28_1.pkl')
URL_list_unlabel.to_csv('URL_canonical_fromblob_jan28_1')

In [157]:
import time
def from_azureBlob_to_table(blob_service_name):
    '''Create a list and fill it with the database dictionary values.
    Return a pd DataFrame'''
    
    blob_items = [] #this is the buffer
    chunk_size  = 1000
    chunk = 0 #this is the chunk number
    i = 10000
    for (i,blob) in enumerate(blob_service_name): # i is the counter

        b = block_blob_service.get_blob_to_text('htmlarchive', blob.name)
        
        if (b.content == None):
            print("b content")
        else:
            soup = BeautifulSoup(b.content, 'html.parser')
    
        if ( (len(soup.find_all('p')) < 3) or (soup.find_all('p')[2].get_text() == None) ):
            print("no content")
        else:
            ads = soup.find_all('p')[2].get_text()

        blob_items.append(ads)
        
        #print(i,ads)
        if i % chunk_size == 0:
            table = pd.DataFrame(blob_items)
            table.to_csv('ads_fromblob_3.{}.csv'.format(chunk))
            b=0
            soup=0
            ads=0
            limit= 0
            blob_items=[]
            chunk +=1
            print('chunk #',chunk,' is done')
            #time.sleep(5)
    return table


In [17]:
def from_azureBlob_to_URLlink(blob_service_name):
    '''Create a list and fill it with the database dictionary values.
    Return a pd DataFrame'''
    
    blob_items = [] #this is the buffer
    chunk_size  = 10000
    chunk = 0 #this is the chunk number

    for (i,blob) in enumerate(blob_service_name): # i is the counter

        b = block_blob_service.get_blob_to_text('htmlarchive', blob.name)
        
        if (b.content == None):
            print("b content empty")
        else:
            soup = BeautifulSoup(b.content, 'html.parser')
    
        if ( (len(soup.findAll('link', rel='canonical')) < 1) ):
            print("no content")
        else:
            links = soup.findAll('link', rel='canonical')
        
        for link in links:
            blob_items.append(link['href'])
        
        #print(i,ads)
        if i % chunk_size == 0:
            print(i)
            links = pd.DataFrame(blob_items)
            links.to_csv('ads_fromblob_URL_only.{}.csv'.format(chunk))
            print('file done: ads_fromblob_URL_only.{}.csv'.format(chunk))
            chunk +=1
            print('chunk #',chunk,' is done for',i,'links')
            #time.sleep(5)


In [ ]:
tableAds=from_azureBlob_to_table(generator)

In [ ]:
blob_df = pd.DataFrame(blob_list)


In [ ]:
blob_df.columns = ['ads']

In [ ]:
blob_df.shape

## Download the blobs

get_blob_to_path(container_name, blob_name, file_path, open_mode='wb', snapshot=None, start_range=None, end_range=None, validate_content=False, progress_callback=None, max_connections=2, lease_id=None, if_modified_since=None, if_unmodified_since=None, if_match=None, if_none_match=None, timeout=None) 

Parameters
container_name
str
Name of existing container.
blob_name
str
Name of existing blob.
file_path
str
Path of file to write out to.
open_mode
str
Mode to use when opening the file. Note that specifying append only open_mode prevents parallel download. So, max_connections must be set to 1 if this open_mode is used.
default value: wb
snapshot
str
The snapshot parameter is an opaque DateTime value that, when present, specifies the blob snapshot to retrieve.
default value: None
start_range
int
Start of byte range to use for downloading a section of the blob. If no end_range is given, all bytes after the start_range will be downloaded. The start_range and end_range params are inclusive. Ex: start_range=0, end_range=511 will download first 512 bytes of blob.
default value: None
end_range
int
End of byte range to use for downloading a section of the blob. If end_range is given, start_range must be provided. The start_range and end_range params are inclusive. Ex: start_range=0, end_range=511 will download first 512 bytes of blob.

## Get Blob to text

get_blob_to_text(container_name, blob_name, encoding='utf-8', snapshot=None, start_range=None, end_range=None, validate_content=False, progress_callback=None, max_connections=2, lease_id=None, if_modified_since=None, if_unmodified_since=None, if_match=None, if_none_match=None, timeout=None) 

Downloads a blob as unicode text, with automatic chunking and progress notifications. Returns an instance of Blob with properties, metadata, and content.
Python

Copy
get_blob_to_text(container_name, blob_name, encoding='utf-8', snapshot=None, start_range=None, end_range=None, validate_content=False, progress_callback=None, max_connections=2, lease_id=None, if_modified_since=None, if_unmodified_since=None, if_match=None, if_none_match=None, timeout=None)
Parameters
container_name
str
Name of existing container.
blob_name
str
Name of existing blob.
encoding
str
Python encoding to use when decoding the blob data.
default value: utf-8
snapshot
str
The snapshot parameter is an opaque DateTime value that, when present, specifies the blob snapshot to retrieve.
default value: None
start_range
int
Start of byte range to use for downloading a section of the blob. If no end_range is given, all bytes after the start_range will be downloaded. The start_range and end_range params are inclusive. Ex: start_range=0, end_range=511 will download first 512 bytes of blob.
default value: None
end_range
int
End of byte range to use for downloading a section of the blob. If end_range is given, start_range must be provided. The start_range and end_range params are inclusive. Ex: start_range=0, end_range=511 will download first 512 bytes of blob.

# Get the table

Check if the table exist and call the generator

In [159]:
from azure.cosmosdb.table.tableservice import TableService
from azure.cosmosdb.table.models import Entity
table_service = TableService(account_name='********', account_key='SzyD**********************2Uw==;')
Ads_bool = table_service.exists('CrawlerAds')
Ads = table_service.query_entities('CrawlerAds')#generator

Create a function to retrive information from the table stored in Azure. 

Then add all those in one big dataframe

In [ ]:
#from larry
def from_azure_to_table(table_service_name):
    '''Create a list and fill it with the database dictionary values.
    Return a pd DataFrame'''
    chunk_size  = 10000
    ads_items = [] #this is the buffer
    chunk = 0 #this is the chunk number
    for (i,a) in enumerate(table_service_name): # i is the counter
        ads_items.append(a) # save the data in buffer
        if i % chunk_size == 0: # after one chunck is done
            table = pd.DataFrame(ads_items)
            table.to_csv('ads.{}.csv'.format(chunk))
            ads_items=[]
            chunk +=1
            print('chunk #',chunk,' is done')

    return table

In [ ]:
tablePhone=from_azure_to_table(Ads)

In [ ]:
one = pd.read_csv('ads.0.csv')


In [ ]:
!ls /Users/cristiana/Desktop/Insight_project/UnlabelData_csv/ads.1*.csv


In [ ]:
#unlabeled 
unlabeled1 = !ls /Users/cristiana/Desktop/Insight_project/UnlabelData_csv/ads.1*.csv

In [ ]:
unlabeled1

In [ ]:
unlabeled2 = !ls /Users/cristiana/Desktop/Insight_project/UnlabelData_csv/ads.2*.csv

In [ ]:
unlabeled3 = !ls /Users/cristiana/Desktop/Insight_project/UnlabelData_csv/ads.3*.csv
unlabeled4 = !ls /Users/cristiana/Desktop/Insight_project/UnlabelData_csv/ads.4*.csv
unlabeled5 = !ls /Users/cristiana/Desktop/Insight_project/UnlabelData_csv/ads.5*.csv
unlabeled6 = !ls /Users/cristiana/Desktop/Insight_project/UnlabelData_csv/ads.6*.csv
unlabeled7 = !ls /Users/cristiana/Desktop/Insight_project/UnlabelData_csv/ads.7*.csv
unlabeled8 = !ls /Users/cristiana/Desktop/Insight_project/UnlabelData_csv/ads.8*.csv
unlabeled9 = !ls /Users/cristiana/Desktop/Insight_project/UnlabelData_csv/ads.9*.csv

In [ ]:
def create_df(listoffile):
    list_ = []

    for file_ in listoffile:
        Allunlabeled = pd.read_csv(file_,index_col=None, header=0)
    
        list_.append(Allunlabeled)

    df_name= pd.concat(list_, axis = 0, ignore_index = True, sort=False)
    return df_name


In [ ]:
unlab_df1 = create_df( unlabeled1 )
unlab_df2 = create_df(unlabeled2 )
unlab_df3 = create_df(unlabeled3)
unlab_df4 = create_df(unlabeled4)
unlab_df5 = create_df(unlabeled5)
unlab_df6 = create_df(unlabeled6)
unlab_df7 = create_df(unlabeled7)
unlab_df8 = create_df(unlabeled8)
unlab_df9 = create_df(unlabeled9)


In [ ]:
unlab_df1.to_pickle('unlable_rawdata_jan27_1.pkl')
unlab_df2.to_pickle('unlable_rawdata_jan27_2.pkl')
unlab_df3.to_pickle('unlable_rawdata_jan27_3.pkl')
unlab_df4.to_pickle('unlable_rawdata_jan27_4.pkl')
unlab_df5.to_pickle('unlable_rawdata_jan27_5.pkl')
unlab_df6.to_pickle('unlable_rawdata_jan27_6.pkl')
unlab_df7.to_pickle('unlable_rawdata_jan27_7.pkl')
unlab_df8.to_pickle('unlable_rawdata_jan27_8.pkl')
unlab_df9.to_pickle('unlable_rawdata_jan27_9.pkl')


In [3]:
unlab_df1['PhoneNumbers'] = unlab_df1['PhoneNumbersJSON'].str.strip('\[\"1').str.strip('\"\]')
unlab_df2['PhoneNumbers'] = unlab_df2['PhoneNumbersJSON'].str.strip('\[\"1').str.strip('\"\]')
unlab_df3['PhoneNumbers'] = unlab_df3['PhoneNumbersJSON'].str.strip('\[\"1').str.strip('\"\]')
unlab_df4['PhoneNumbers'] = unlab_df4['PhoneNumbersJSON'].str.strip('\[\"1').str.strip('\"\]')
unlab_df5['PhoneNumbers'] = unlab_df5['PhoneNumbersJSON'].str.strip('\[\"1').str.strip('\"\]')
unlab_df6['PhoneNumbers'] = unlab_df6['PhoneNumbersJSON'].str.strip('\[\"1').str.strip('\"\]')
unlab_df7['PhoneNumbers'] = unlab_df7['PhoneNumbersJSON'].str.strip('\[\"1').str.strip('\"\]')
unlab_df8['PhoneNumbers'] = unlab_df8['PhoneNumbersJSON'].str.strip('\[\"1').str.strip('\"\]')
unlab_df9['PhoneNumbers'] = unlab_df9['PhoneNumbersJSON'].str.strip('\[\"1').str.strip('\"\]')


In [4]:
unlab_df1['PhoneNumbers'].fillna(value = 0, inplace = True)
unlab_df2['PhoneNumbers'].fillna(value = 0, inplace = True)
unlab_df3['PhoneNumbers'].fillna(value = 0, inplace = True)
unlab_df4['PhoneNumbers'].fillna(value = 0, inplace = True)
unlab_df5['PhoneNumbers'].fillna(value = 0, inplace = True)
unlab_df6['PhoneNumbers'].fillna(value = 0, inplace = True)
unlab_df7['PhoneNumbers'].fillna(value = 0, inplace = True)
unlab_df8['PhoneNumbers'].fillna(value = 0, inplace = True)
unlab_df9['PhoneNumbers'].fillna(value = 0, inplace = True)


In [5]:
list1 = unlab_df1['PhoneNumbers'].tolist()
list2 = unlab_df2['PhoneNumbers'].tolist()
list3 = unlab_df3['PhoneNumbers'].tolist()
list4 = unlab_df4['PhoneNumbers'].tolist()
list5 = unlab_df5['PhoneNumbers'].tolist()
list6 = unlab_df6['PhoneNumbers'].tolist()
list7 = unlab_df7['PhoneNumbers'].tolist()
list8 = unlab_df8['PhoneNumbers'].tolist()
list9 = unlab_df9['PhoneNumbers'].tolist()


In [11]:
list_tot_phone = list1 + list2 + list3 + list4 +list5 + list6 +list7+ list8 +list9

In [192]:
print( len(list1), len(list2), len(list3), len(list4), len(list5), len(list6), len(list7), len(list8), len(list9), len(list_tot_phone) )


1110000 1110000 350000 110000 110000 110000 110000 110000 110000 3230000


In [12]:
len(list_tot_phone)

3230000

In [15]:
list_tot_phone = [x for x in list_tot_phone if x != 0]

In [16]:
len(list_tot_phone)

2487317

## Positive label data (phone numbers)

In [17]:
positive = pd.read_excel('/Users/cristiana/Desktop/Insight_project/Positive_Phone_Contact_Seattle_Region.xlsx',skiprows = 2)


In [18]:
positive2 = pd.read_csv('/Users/cristiana/Desktop/Insight_project/potential-victims-Hannahremoved1.csv',usecols=["Full list confirmed phone numbers", "removed the 1"] )

Put the phone numbers in two lists and see if matches

In [19]:
positive2.columns = ['PhoneNumberCOM','PhoneNumbers']
positive2.head()


,PhoneNumberCOM,PhoneNumbers
0,12018972064,2018972064
1,12026307299,2026307299
2,12027530440,2027530440
3,12036466852,2036466852
4,12052121570,2052121570


In [38]:
pos = positive["Phone Number"].tolist()
pos2 = positive2["PhoneNumbers"].tolist()

In [41]:
pos2  = [str(i) for i in pos2]


In [42]:
newpos = [i.replace('-','') for i in pos]


In [43]:
tot_double2 = [x for x in newpos if x in list_tot_phone] 

In [44]:
tot_double = [x for x in pos2 if x in list_tot_phone] 

In [45]:
match_list = tot_double + tot_double2

In [46]:
print(len(newpos),len(pos2),len(list_tot_phone),len(tot_double),len(tot_double2), len( match_list  ))

65 1145 2487317 755 37 792


In [47]:
Match_positive = pd.DataFrame(match_list)

In [48]:
Match_positive.to_pickle('Phone_number_matching_only37_jan28.pkl')
Match_positive.to_csv('Phone_number_matching.csv')

# extract unlabel that match label

In [52]:
def check_match(list_name,dataframe):
    list_lab = []
    for item in list_name:
        Labeled = dataframe[dataframe['PhoneNumbers'].str.contains(item, na=False)]
        list_lab.append(Labeled)
        df_name_lab= pd.concat(list_lab, axis = 0, ignore_index = True, sort=False)
    print('Dtaframe '+str(dataframe)+' done!')
    return df_name_lab

In [53]:
match1 = check_match(match_list, unlab_df1)
match2 = check_match(match_list, unlab_df2)
match3 = check_match(match_list, unlab_df3)
match4 = check_match(match_list, unlab_df4)
match5 = check_match(match_list, unlab_df5)
match6 = check_match(match_list, unlab_df6)
match7 = check_match(match_list, unlab_df7)
match8 = check_match(match_list, unlab_df8)
match9 = check_match(match_list, unlab_df9)

Dtaframe          Unnamed: 0                    CrawlStartTime  \
0        0           2018-08-12 15:00:19.599256+00:00   
1        1           2018-08-12 15:00:19.599256+00:00   
2        2           2018-08-12 15:00:19.599256+00:00   
3        3           2018-08-12 15:00:19.599256+00:00   
4        4           2018-08-12 15:00:19.599256+00:00   
5        5           2018-08-12 15:00:19.599256+00:00   
6        6           2018-08-12 15:00:19.599256+00:00   
7        7           2018-08-12 15:00:19.599256+00:00   
8        8           2018-08-12 15:00:19.599256+00:00   
9        9           2018-08-12 15:00:19.599256+00:00   
10       10          2018-08-12 15:00:19.599256+00:00   
11       11          2018-08-12 15:00:19.599256+00:00   
12       12          2018-08-12 15:00:19.599256+00:00   
13       13          2018-08-12 15:00:19.599256+00:00   
14       14          2018-08-12 15:00:19.599256+00:00   
15       15          2018-08-12 15:00:19.599256+00:00   
16       16          2

Dtaframe          Unnamed: 0                    CrawlStartTime  \
0        0           2019-01-25 01:00:21.607905+00:00   
1        1           2019-01-25 01:00:21.607905+00:00   
2        2           2019-01-25 01:00:21.607905+00:00   
3        3           2019-01-25 01:00:21.607905+00:00   
4        4           2019-01-25 01:00:21.607905+00:00   
5        5           2019-01-25 01:00:21.607905+00:00   
6        6           2019-01-25 01:00:21.607905+00:00   
7        7           2019-01-25 01:00:21.607905+00:00   
8        8           2019-01-25 01:00:21.607905+00:00   
9        9           2019-01-25 01:00:21.607905+00:00   
10       10          2019-01-25 01:00:21.607905+00:00   
11       11          2019-01-25 01:00:21.607905+00:00   
12       12          2019-01-25 01:00:21.607905+00:00   
13       13          2019-01-25 01:00:21.607905+00:00   
14       14          2019-01-25 01:00:21.607905+00:00   
15       15          2019-01-25 01:00:21.607905+00:00   
16       16          2

Dtaframe         Unnamed: 0                    CrawlStartTime  \
0       0           2019-01-25 01:00:21.607905+00:00   
1       1           2019-01-25 01:00:21.607905+00:00   
2       2           2019-01-25 01:00:21.607905+00:00   
3       3           2019-01-25 01:00:21.607905+00:00   
4       4           2019-01-25 01:00:21.607905+00:00   
5       5           2019-01-25 01:00:21.607905+00:00   
6       6           2019-01-25 01:00:21.607905+00:00   
7       7           2019-01-25 01:00:21.607905+00:00   
8       8           2019-01-25 01:00:21.607905+00:00   
9       9           2019-01-25 01:00:21.607905+00:00   
10      10          2019-01-25 01:00:21.607905+00:00   
11      11          2019-01-25 01:00:21.607905+00:00   
12      12          2019-01-25 01:00:21.607905+00:00   
13      13          2019-01-25 01:00:21.607905+00:00   
14      14          2019-01-25 01:00:21.607905+00:00   
15      15          2019-01-25 01:00:21.607905+00:00   
16      16          2019-01-25 01:00:21

Dtaframe         Unnamed: 0                    CrawlStartTime  \
0       0           2019-01-25 01:00:21.607905+00:00   
1       1           2019-01-25 01:00:21.607905+00:00   
2       2           2019-01-25 01:00:21.607905+00:00   
3       3           2019-01-25 01:00:21.607905+00:00   
4       4           2019-01-25 01:00:21.607905+00:00   
5       5           2019-01-25 01:00:21.607905+00:00   
6       6           2019-01-25 01:00:21.607905+00:00   
7       7           2019-01-25 01:00:21.607905+00:00   
8       8           2019-01-25 01:00:21.607905+00:00   
9       9           2019-01-25 01:00:21.607905+00:00   
10      10          2019-01-25 01:00:21.607905+00:00   
11      11          2019-01-25 01:00:21.607905+00:00   
12      12          2019-01-25 01:00:21.607905+00:00   
13      13          2019-01-25 01:00:21.607905+00:00   
14      14          2019-01-25 01:00:21.607905+00:00   
15      15          2019-01-25 01:00:21.607905+00:00   
16      16          2019-01-25 01:00:21

Dtaframe         Unnamed: 0                    CrawlStartTime  \
0       0           2019-01-25 01:00:21.607905+00:00   
1       1           2019-01-25 01:00:21.607905+00:00   
2       2           2019-01-25 01:00:21.607905+00:00   
3       3           2019-01-25 01:00:21.607905+00:00   
4       4           2019-01-25 01:00:21.607905+00:00   
5       5           2019-01-25 01:00:21.607905+00:00   
6       6           2019-01-25 01:00:21.607905+00:00   
7       7           2019-01-25 01:00:21.607905+00:00   
8       8           2019-01-25 01:00:21.607905+00:00   
9       9           2019-01-25 01:00:21.607905+00:00   
10      10          2019-01-25 01:00:21.607905+00:00   
11      11          2019-01-25 01:00:21.607905+00:00   
12      12          2019-01-25 01:00:21.607905+00:00   
13      13          2019-01-25 01:00:21.607905+00:00   
14      14          2019-01-25 01:00:21.607905+00:00   
15      15          2019-01-25 01:00:21.607905+00:00   
16      16          2019-01-25 01:00:21

Dtaframe         Unnamed: 0                    CrawlStartTime  \
0       0           2019-01-25 01:00:21.607905+00:00   
1       1           2019-01-25 01:00:21.607905+00:00   
2       2           2019-01-25 01:00:21.607905+00:00   
3       3           2019-01-25 01:00:21.607905+00:00   
4       4           2019-01-25 01:00:21.607905+00:00   
5       5           2019-01-25 01:00:21.607905+00:00   
6       6           2019-01-25 01:00:21.607905+00:00   
7       7           2019-01-25 01:00:21.607905+00:00   
8       8           2019-01-25 01:00:21.607905+00:00   
9       9           2019-01-25 01:00:21.607905+00:00   
10      10          2019-01-25 01:00:21.607905+00:00   
11      11          2019-01-25 01:00:21.607905+00:00   
12      12          2019-01-25 01:00:21.607905+00:00   
13      13          2019-01-25 01:00:21.607905+00:00   
14      14          2019-01-25 01:00:21.607905+00:00   
15      15          2019-01-25 01:00:21.607905+00:00   
16      16          2019-01-25 01:00:21

Dtaframe         Unnamed: 0                    CrawlStartTime  \
0       0           2018-08-23 01:00:27.028162+00:00   
1       1           2018-08-23 01:00:27.028162+00:00   
2       2           2018-08-23 01:00:27.028162+00:00   
3       3           2018-08-23 01:00:27.028162+00:00   
4       4           2018-08-23 01:00:27.028162+00:00   
5       5           2018-08-23 01:00:27.028162+00:00   
6       6           2018-08-23 01:00:27.028162+00:00   
7       7           2018-08-23 01:00:27.028162+00:00   
8       8           2018-08-23 01:00:27.028162+00:00   
9       9           2018-08-23 01:00:27.028162+00:00   
10      10          2018-08-23 01:00:27.028162+00:00   
11      11          2018-08-23 01:00:27.028162+00:00   
12      12          2018-08-23 01:00:27.028162+00:00   
13      13          2018-08-23 01:00:27.028162+00:00   
14      14          2018-08-23 01:00:27.028162+00:00   
15      15          2018-08-23 01:00:27.028162+00:00   
16      16          2018-08-23 01:00:27

Dtaframe         Unnamed: 0                    CrawlStartTime  \
0       0           2018-11-21 01:00:02.879871+00:00   
1       1           2018-11-21 01:00:02.879871+00:00   
2       2           2018-11-29 01:00:01.443485+00:00   
3       3           2018-11-30 01:00:01.396308+00:00   
4       4           2018-12-01 01:00:02.400748+00:00   
5       5           2018-12-02 01:00:02.529246+00:00   
6       6           2018-12-03 01:00:01.453963+00:00   
7       7           2018-12-28 01:00:00.833768+00:00   
8       8           2018-08-23 01:00:27.028162+00:00   
9       9           2019-01-21 01:00:00.586575+00:00   
10      10          2018-12-22 01:00:01.670456+00:00   
11      11          2018-08-12 15:00:19.599256+00:00   
12      12          2018-08-23 01:00:27.028162+00:00   
13      13          2018-08-23 01:00:27.028162+00:00   
14      14          2018-08-23 01:00:27.028162+00:00   
15      15          2018-08-23 01:00:27.028162+00:00   
16      16          2018-08-23 01:00:27

Dtaframe         Unnamed: 0                    CrawlStartTime  \
0       0           2018-08-23 01:00:27.028162+00:00   
1       1           2018-08-23 01:00:27.028162+00:00   
2       2           2018-11-27 01:00:02.298719+00:00   
3       3           2018-08-23 01:00:27.028162+00:00   
4       4           2018-08-23 01:00:27.028162+00:00   
5       5           2018-08-23 01:00:27.028162+00:00   
6       6           2019-01-22 01:00:02.246149+00:00   
7       7           2018-08-23 01:00:27.028162+00:00   
8       8           2018-08-23 01:00:27.028162+00:00   
9       9           2019-01-24 01:00:03.101633+00:00   
10      10          2018-08-23 01:00:27.028162+00:00   
11      11          2018-09-07 01:00:04.032422+00:00   
12      12          2018-10-27 01:00:01.460201+00:00   
13      13          2018-08-12 15:00:19.599256+00:00   
14      14          2018-10-19 01:00:02.328167+00:00   
15      15          2018-10-24 01:00:01.354667+00:00   
16      16          2018-08-23 01:00:27

In [54]:
match1.head()

,Unnamed: 0,CrawlStartTime,DatePosted,ErrorParsing,FullURL,GeneratePotentialVictims,Hostname,ImageURIsJSON,PartitionKey,PhoneNumbersJSON,PotentialVictimId,Region,RowKey,Timestamp,Type,etag,PhoneNumbers
0,6633,2018-08-23 01:00:27.028162+00:00,2018-08-29 14:32:08.511698+00:00,False,https://www.cityxguide.com/escorts/b-ty-l-vers-1-choice-j-ic-grip-mind-b-l-o-n-g-lips-and-toe-curling-tri-577104,False,www.cityxguide.com,"[""https://images.cityxguide.com/20161004044227/0000/0000/0266/6629/1f2b9cdd6e43a11450ee5b4d82521f99-m.jpg"",""https://images.cityxguide.com/20161004044227/0000/0000/0266/6678/731ac6297acae22fd3ed792016143e65-m.jpg"",""https://images.cityxguide.com/20161004044227/0000/0000/0266/6726/b3f86380d85e6431cde6f162c5c903d3-m.jpg"",""https://images.cityxguide.com/20161004044227/0000/0000/0266/6778/85052809b30b347a688f7ee4ef3b698f-m.jpg"",""https://images.cityxguide.com/20161004044227/0000/0000/0266/6822/30caec2544ad9290ce8cc251e0ae2394-m.jpg"",""https://images.cityxguide.com/20161004044227/0000/0000/0266/6864/4133ef42a6f7eec6870c104b3582e631-m.jpg"",""https://images.cityxguide.com/20161004044227/0000/0000/0266/6910/f7157f52d9961919b1787cc66d5d2ed7-m.jpg"",""https://images.cityxguide.com/20161004044227/0000/0000/0266/6959/29bfabd698845bb1cae8848313f8f5a6-m.jpg""]",www.cityxguide.com,"[""12036466852""]",NaN,"Seattle, Washington",https:||www.cityxguide.com|escorts|b-ty-l-vers-1-choice-j-ic-grip-mind-b-l-o-n-g-lips-and-toe-curling-tri-577104,2018-08-29 14:32:08.570711+00:00,ad,"W/""datetime'2018-08-29T14%3A32%3A08.5707115Z'""",2036466852
1,6634,2018-08-23 01:00:27.028162+00:00,2018-08-29 14:32:43.852718+00:00,False,https://www.cityxguide.com/escorts/b-ty-l-vers-1-choice-j-ic-grip-mind-b-l-o-n-g-lips-and-toe-curling-tri__1-586139,False,www.cityxguide.com,"[""https://images.cityxguide.com/20161004044227/0000/0000/0270/7622/41275bdad33f6181db2572c508dd8982-m.jpg"",""https://images.cityxguide.com/20161004044227/0000/0000/0270/7654/8ae4aa0d1e7a2db2b1cb5a2256750619-m.jpg"",""https://images.cityxguide.com/20161004044227/0000/0000/0270/7679/8b3f5d8dc2bc59ce71fbeaa953913030-m.jpg"",""https://images.cityxguide.com/20161004044227/0000/0000/0270/7705/41c2d1134f39637cbf3aef03afe6cca0-m.jpg"",""https://images.cityxguide.com/20161004044227/0000/0000/0270/7734/0711e9181f9d1ac43e7ec82836d23981-m.jpg"",""https://images.cityxguide.com/20161004044227/0000/0000/0270/7762/b8b5c0d9d5a00148e4799a6518a54764-m.jpg"",""https://images.cityxguide.com/20161004044227/0000/0000/0270/7791/004ecae171270be59bed9dad24952252-m.jpg"",""https://images.cityxguide.com/20161004044227/0000/0000/0270/7818/4b293cae9e83ac55039bbb248678baaf-m.jpg""]",www.cityxguide.com,"[""12036466852""]",NaN,"Seattle, Washington",https:||www.cityxguide.com|escorts|b-ty-l-vers-1-choice-j-ic-grip-mind-b-l-o-n-g-lips-and-toe-curling-tri__1-586139,2018-08-29 14:32:43.886807+00:00,ad,"W/""datetime'2018-08-29T14%3A32%3A43.8868071Z'""",2036466852
2,3215,2018-08-23 01:00:27.028162+00:00,2018-08-24 05:46:21.945502+00:00,False,https://www.cityxguide.com/escorts/big-booty-babe-z-g-z-nds-down-t-e-best__2-7454419,False,www.cityxguide.com,"[""https://images.cityxguide.com/20161004044227/0000/0000/1312/6098/074a65fb321de080876ead8ef43ef79d-m.jpg"",""https://images.cityxguide.com/20161004044227/0000/0000/1312/6103/a7048e116dac94d4e3ad623bd16db16f-m.jpg"",""https://images.cityxguide.com/20161004044227/0000/0000/1312/6106/5754b185d409bb1f0532a0ff27dcca09-m.jpg""]",www.cityxguide.com,"[""12036466852""]",NaN,"Las Vegas, Nevada",https:||www.cityxguide.com|escorts|big-booty-babe-z-g-z-nds-down-t-e-best__2-7454419,2018-08-24 05:46:21.982965+00:00,ad,"W/""datetime'2018-08-24T05%3A46%3A21.9829657Z'""",2036466852
3,6742,2018-08-23 01:00:27.028162+00:00,2018-08-27 06:21:42.410486+00:00,False,https://www.cityxguide.com/escorts/big-booty-e-tic-amazing-curvaceous-100-real-f-r-e-a-k__14-2345801,False,www.cityxguide.com,"[""https://images.cityxguide.com/20161004044227/0000/0000/1038/1122/27e9520ba4f17961158bf64ac0906e93-

In [55]:
match1.shape

(3332, 17)

In [56]:
match_tot1 = match1.append(match2,sort=False)
match_tot2 = match_tot1.append(match3,sort=False)
match_tot3 = match_tot2.append(match4,sort=False)
match_tot4 = match_tot3.append(match5,sort=False)
match_tot5 = match_tot4.append(match6,sort=False)
match_tot6 = match_tot5.append(match7,sort=False)
match_tot7 = match_tot6.append(match8,sort=False)
match_tot8 = match_tot7.append(match9,sort=False)

In [57]:
match_tot8.shape

(11892, 17)

In [58]:
match_tot8.to_pickle('Phone_number_matching_url_jan28.pkl')
match_tot8.to_csv('Phone_number_matching_url_28jan.csv')

In [59]:
match_tot8['FullURL'].head()

0    https://www.cityxguide.com/escorts/b-ty-l-vers-1-choice-j-ic-grip-mind-b-l-o-n-g-lips-and-toe-curling-tri-577104   
1    https://www.cityxguide.com/escorts/b-ty-l-vers-1-choice-j-ic-grip-mind-b-l-o-n-g-lips-and-toe-curling-tri__1-586139
2    https://www.cityxguide.com/escorts/big-booty-babe-z-g-z-nds-down-t-e-best__2-7454419                               
3    https://www.cityxguide.com/escorts/big-booty-e-tic-amazing-curvaceous-100-real-f-r-e-a-k__14-2345801               
4    https://www.cityxguide.com/escorts/big-booty-e-tic-amazing-curvaceous-100-real-f-r-e-a-k__23-5905273               
Name: FullURL, dtype: object

In [72]:
urls_list_un = match_tot8['FullURL'].tolist()

In [61]:
print(len(urls_list_un))

11892


In [62]:
urls_list_un = set(urls_list_un)

In [63]:
print(len(urls_list_un))

11426
